In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
import random

In [3]:
import numpy as np

In [4]:
from PIL import Image

In [5]:
import zipfile

In [6]:
f=zipfile.ZipFile('./CT_COVID.zip','r')
for file in f.namelist():
    f.extract(file,"./CT_COVID") #解压到的位置，./表示当前目录(与此.ipynb文件同一个目录)
f.close()

In [7]:
f=zipfile.ZipFile('./CT_NonCOVID.zip','r')
for file in f.namelist():
    f.extract(file,"./CT_NonCOVID") #解压到的位置，./表示当前目录(与此.ipynb文件同一个目录)
f.close()

In [8]:
import os
import cv2

In [9]:
def read_directory(directory_name):
    array_of_img=[]
    for filename in os.listdir(r"./"+directory_name):
        img = cv2.imread(directory_name + "/" + filename)
        array_of_img.append(img)
    return array_of_img


In [10]:
CT_COVID=read_directory('./CT_COVID')

In [11]:
CT_NonCOVID=read_directory('./CT_NonCOVID')

In [12]:
#未得肺炎标记为0，否则标记为1
list_covid=[]
list_noncovid=[]
for i in range(len(CT_COVID)):
    list_covid.append(1)
for i in range(len(CT_NonCOVID)):
    list_noncovid.append(0)
    

In [13]:
CT_train1=CT_COVID[0:300]+CT_NonCOVID[0:300]
CT_test1=CT_COVID[300:]+CT_NonCOVID[300:]

In [14]:
CT_train_label=list_covid[0:300]+list_noncovid[0:300]
CT_test_label=list_covid[300:]+list_noncovid[300:]

In [15]:
CT_train=[]
for i in CT_train1:
    i=np.resize(i,(3,400,400))
    i=torch.tensor(i)
    i=i.reshape(1,3,400,400)
    CT_train.append(i)

In [16]:
CT_test=[]
for i in CT_train1:
    i=np.resize(i,(3,400,400))
    i=torch.tensor(i)
    i=i.reshape(1,3,400,400)
    CT_test.append(i)

In [17]:
train_pic_label=[]
for i,j in zip(CT_train,CT_train_label):
    j=torch.tensor([j])
    train_pic_label.append((i,j))
test_pic_label=[]
for i,j in zip(CT_test,CT_test_label):
    j=torch.tensor([j])
    test_pic_label.append((i,j))

In [18]:
#乱序
random.shuffle(train_pic_label)
random.shuffle(test_pic_label)

In [23]:
CT_train3=[]
CT_train3_label=[]
for i,j in train_pic_label:
    i=i.numpy()
    CT_train3.append(i)
    CT_train3_label.append(j)
    

In [35]:
list_train_label=[]
for i in range(10):
    group=CT_train3[i*60:(i+1)*60] 
    group_label=CT_train3_label[i*60:(i+1)*60]
    sample_label=torch.tensor(group_label).reshape(1,60)
    sample=np.array(group).reshape(60,3,400,400)
    sample=torch.tensor(sample)
    list_train_label.append((sample,sample_label))
    

In [44]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19,self).__init__()
        self.conv1_1=nn.Conv2d(3,64,kernel_size=3)
        self.conv1_2=nn.Conv2d(64,64,kernel_size=3)

        self.conv2_1=nn.Conv2d(64,128,kernel_size=3)
        self.conv2_2=nn.Conv2d(128,128,kernel_size=3)

        self.conv3_1=nn.Conv2d(128,256,kernel_size=3)
        self.conv3_2=nn.Conv2d(256,256,kernel_size=3)
        self.conv3_3=nn.Conv2d(256,256,kernel_size=3)
        self.conv3_4=nn.Conv2d(256,256,kernel_size=3)

        self.conv4_1=nn.Conv2d(256,512,kernel_size=3)
        self.conv4_2=nn.Conv2d(512,512,kernel_size=3)
        self.conv4_3=nn.Conv2d(512,512,kernel_size=3)
        self.conv4_4=nn.Conv2d(512,512,kernel_size=3)

        self.conv5_1=nn.Conv2d(512,512,kernel_size=3)
        self.conv5_2=nn.Conv2d(512,512,kernel_size=3)
        self.conv5_3=nn.Conv2d(512,512,kernel_size=3)
        self.conv5_4=nn.Conv2d(512,512,kernel_size=3)

        self.conv5_drop=nn.Dropout2d()

        self.fc1=nn.Linear(12800,4096)
        self.fc2=nn.Linear(4096,4096)
        self.fc3=nn.Linear(4096,2)
    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1_2(self.conv1_1(x)),2))
        x=F.relu(F.max_pool2d(self.conv2_2(self.conv2_1(x)),2))
        x=F.relu(F.max_pool2d(self.conv3_4(self.conv3_3(self.conv3_2(self.conv3_1(x)))),2))
        x=F.relu(F.max_pool2d(self.conv4_4(self.conv4_3(self.conv4_2(self.conv4_1(x)))),2))
        x=F.relu(F.max_pool2d(self.conv5_drop(self.conv5_3(self.conv5_4(self.conv5_2(self.conv5_1(x))))),2)) 
        x=x.view(-1,12800)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=F.relu(self.fc2(x))
        x=F.dropout(x,training=self.training)
        x=F.relu(self.fc3(x))
        return F.softmax(x,dim=1)

In [45]:
model=VGG19()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.005,momentum=0.9)

In [46]:
cuda_gpu=torch.cuda.is_available()

In [47]:
def train(model,criterion,optimizer,dataset):
    model.train()
    for data,target in dataset:
        if cuda_gpu:
            data,target=data.cuda(),target.cuda()
            model.cuda()
        data,target=Variable(data).float(),Variable(target)
        output=model(data)
        optimizer.zero_grad()
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()

In [48]:
train(model,criterion,optimizer,train_pic_label)

In [49]:
def test(model,epoch,criterion,data_loader):
    model.eval()
    test_loss=0
    correct=0
    for data,target in data_loader:
        if cuda_gpu:
            data,target=data.cuda(),target.cuda()
            model.cuda()
        data,target=Variable(data).float(),Variable(target)
        output=model(data)
        test_loss+=criterion(output,target).item()
        pred=output.data.max(1)[1]
        correct+=pred.eq(target.data).cpu().sum()
    test_loss/=len(data_loader)
    acc=correct/len(data_loader)
    return (acc,test_loss)

In [50]:
test(model,5,criterion,test_pic_label)

(tensor(0.6622), 0.6931471824645996)